In [ ]:
# Imports
import glob
import os
from PIL import Image
import numpy as np
import pandas as pd
import scipy.signal

# Image processing tools
import skimage.feature
import skimage.filters
import skimage.filters.rank
import skimage.io
import skimage.morphology
import skimage.segmentation
from scipy import ndimage as ndi

import bebi103

import bokeh
from bokeh.models.widgets import Panel, Tabs
from bokeh.io import output_file, show
from bokeh.plotting import figure

bokeh.io.output_notebook()

In [ ]:
def show_two_ims(im_1, im_2, titles=[None, None], interpixel_distances=[0.13, 0.13],
                 color_mapper=None):
    """Convenient function for showing two images side by side."""
    
    p_1 = bebi103.viz.imshow(im_1,
                             plot_height=600,
                             title=titles[0],
                             color_mapper=color_mapper,
                             interpixel_distance=interpixel_distances[0],
                             length_units='µm')
   
    p_2 = bebi103.viz.imshow(im_2,
                             plot_height=600,
                             title=titles[1],
                             color_mapper=color_mapper,
                             interpixel_distance=interpixel_distances[0],
                             length_units='µm')
    
    p_2.x_range = p_1.x_range
    p_2.y_range = p_1.y_range
    
    return bokeh.layouts.gridplot([p_1, p_2], ncols=2)

In [ ]:
def thresh_finder(second_derivatives, bins, desired_thresh = 2000):
    ''' 
    Searches through a list of second derivatives for the bin value at which the
    previous derivative was greater than desired thresh and the current derivative is less
    than the desired thresh. If there are multiple events where this occurs, they are stored in a list and the largest one is used as a threshold. 
    '''
    
    past_i = 0
    index = 0
    thresh = []
    
    # Loop through the second derivatives 
    for i in second_derivatives:
        
        # Look for the threshold location
        if past_i > desired_thresh and i < desired_thresh:
        
            # Interpolate to find the appropriate bin equal to the desired thresh
            threshold = bins[index-1] + (bins[index] - bins[index-1])*(desired_thresh - past_i)/(i-past_i)
            thresh.append(threshold)
        
        # Reset the parameters
        past_i = i
        index += 1

    return(np.max(thresh))

In [ ]:
def remove_large_objects(im, max_size = 1000, connectivity = 1):
    ''' Function to remove large objects in an image. Useful for removing large autofluorescent features.
    Parameters:
    im: a binary image array to remove the large objects from
    max_size: the maximum sized object to keep in the image
    connectivity: required pixel connectivity for contiguity
    
    Returns: 
    out: Binary image array with large ojects removed
    '''
    
    out = im.copy()
    
    selem = ndi.generate_binary_structure(im.ndim, connectivity)
    ccs = np.zeros_like(im, dtype=np.int32)
    ndi.label(im, selem, output=ccs)

    component_sizes = np.bincount(ccs.ravel())
    
    too_big = component_sizes > max_size
    too_big_mask = too_big[ccs]
    out[too_big_mask] = 0

    return(out)

In [ ]:
def brightness_counter(im_labeled, im_pos, im_height, im_width, template_height, template_width):
    '''
    Brightness counter is a function that produces a list of brightness values for up to one thousand cells in an image. 
    The brightness values is a sum of the intensity for a given labelled object.
    
    Parameters
    im_labeled: An image array with each independant object labeled with a separate value generated by skimage.measure.label
    im_pos: The result of the autofluorescence subtraction
    
    Returns
    cell_list: list of cells selected for analysis
    cell_intensities: list of cell intensities
    '''
    
    # Determine whether down-sampling is required 
    if np.max(im_labeled) > 1000:
        # Create a list of the cells to quantify the brightness in using a random number generator without replacement  
        cell_list = np.random.choice(np.arange(1, np.max(im_labeled)), 1000, replace=False)
    else:
        # If there are less than 1000 cells, create a list of all the cells to access
        cell_list = np.arange(1, np.max(im_labeled) + 1)
    
    cell_intensities = []
    
    # Create the cropped image to ensure that the image arrays are the same size. 
    im_pos_crop = im_pos[int(template_height/2 - 1):int(im_height - int(template_height/2)), int(template_width/2 - 1):int(im_width - int(template_width/2))]
 
    # Loop through the prepared cell list
    for count_value in cell_list:
        # Obtain the index of the current cell
        inds = (im_labeled == count_value)
        
        # In the positive image, sum over the whole cell to get its intensity 
        cell_intensity = np.sum(im_pos_crop[inds])/np.sum(inds)
        
        # Append this to a list of all cell intensities
        cell_intensities.append(cell_intensity)
    
    return(cell_list, cell_intensities)

In [ ]:
def cell_counter(im, template, size_thresh = 0.62, gaussian_size = 5, truncation = 2, threshold = 1000, min_size = 10):
    """
    Produces an cell count for cells transduced by a viral vector in lung tissue.
    
    Parameters
    im: Colored image at least two color channels
    template: an array to perform feature matching with
    size_thresh: minimum similarity requirement for feature matching
    gaussian_size: dictates shape of the gaussian used for gaussian blur and subtraction
    truncation: allows tuning of the gaussian blur.
    """

    im_height = np.shape(im)[0]
    im_width = np.shape(im)[1]
    
    template_height = np.shape(template)[0]
    template_width = np.shape(template)[1]
    
    # Obtain the red (autofluorescent) and green (signal) channels of an image
    im_af = skimage.img_as_float(im[:,:,0])
    im_sig = skimage.img_as_float(im[:,:,1])

    # Subtract the autofluorescence from the signal and determine their mean. Adjust the autofluorescent image by this mean
    # Subtract the autofluorescent image from the signal image 
    im_sub = im_sig - im_af*np.mean(im_sig)/np.mean(im_af)
    
    # Remove all regions of the image less than zero (improving the gaussian blur)
    im_pos = np.where(im_sub<0, 0, im_sub)
    
    im_feat = skimage.feature.match_template(im_pos, template)
    
    binary_feat = im_feat > size_thresh

    binary_feat = skimage.morphology.remove_small_objects(binary_feat, min_size=min_size)

    # Apply a gaussian blur to determine the background patterning 
    im_bg = skimage.filters.gaussian(im_pos, gaussian_size, truncate = truncation)

    im_no_bg = im_pos - im_bg

    im_no_bg = (im_no_bg - im_no_bg.min()) / (im_no_bg.max() - im_no_bg.min())
    
    # Compute the histigram of the image intensities
    hist_bin = skimage.exposure.histogram(im_no_bg)
    hist, bins = hist_bin
    
    # Determine the first derivative of the gradient
    dn_dk = np.diff(np.diff(hist))
    
    thresh = thresh_finder(dn_dk, bins, desired_thresh = threshold)
    
    # Threshold the image based the automated threshold
    binary_gauss = im_no_bg[int(template_height/2 - 1):int(im_height - int(template_height/2)), int(template_width/2 - 1):int(im_width - int(template_width/2))] > thresh

    binary_gauss = skimage.morphology.binary_closing(binary_gauss, skimage.morphology.disk(3))
    
    # binary_gauss = remove_large_objects(binary_gauss)
    
    binary_mult = binary_gauss*binary_feat

    binary_rem = skimage.morphology.remove_small_objects(binary_mult, min_size=min_size)
    
    im_labeled, n_labels = skimage.measure.label(
                            binary_rem, background=0, return_num=True)
    
    cell_list, cell_intensity_list = brightness_counter(im_labeled, im_pos, im_height, im_width, template_height, template_width)
    
    return binary_rem, n_labels, cell_list, cell_intensity_list

In [ ]:
# Start by loading in the template file. 
template = np.load('template.npy')

# Display the template
bokeh.io.show(bebi103.viz.imshow(template, plot_height=900))

In [ ]:
# Load a test image 
fname = 'Lung_Paper_Images/Test_Images/Lung/CAPA4/1_00076_Overlay.tif'

im = skimage.io.imread(fname)

# Run the cell counter on the test image
binary_mask, n_cells, cell_list, cell_brightness_list = cell_counter(im, template)

im_float = skimage.img_as_float(im)[:,:,:2]

im_height = np.shape(im)[0]
im_width = np.shape(im)[1]
    
template_height = np.shape(template)[0]
template_width = np.shape(template)[1]

# Build RGB image, cropping the image by the dimensions of the template
im_p_rgb = np.dstack((im_float[:,:,0][int(template_height/2 - 1):int(im_height - int(template_height/2)), int(template_width/2 - 1):int(im_width - int(template_width/2))], im_float[:,:,1][int(template_height/2 - 1):int(im_height - int(template_height/2)), int(template_width/2 - 1):int(im_width - int(template_width/2))], binary_mask))

#skimage.io.imsave('test.jpg', im_p_rgb, quality = 100)

bokeh.io.show(show_two_ims(im_float[:,:,:2], im_p_rgb[int(template_height/2 - 1):int(im_height - int(template_height/2)), int(template_width/2 - 1):int(im_width - int(template_width/2))], titles=['raw', 'counted'], color_mapper = 'rgb'))

In [ ]:
#Plot the brightness list to 
p = bokeh.plotting.figure(y_axis_label = 'ECDF',
                          x_axis_label = 'Brightness per Cell')

p = bebi103.viz.ecdf(cell_brightness_list, 
                         plot_height=500, 
                         plot_width=1000,
                         line_width = 0.5,
                         p=p)
bokeh.io.show(p)

In [ ]:
# Set all of the required parameters 
    # The primary parameters to change are the size thresh and the threshold. 
    # size_thresh: lung, 0.62; liver, 0.4;
    # threshold: lung, 1000; liver, 10000;
    
size_thresh = 0.62
gaussian_size = 5
truncation = 2
threshold = 1000
min_size = 10

# File path to the images
data_dir = 'Lung_Paper_Images/Pre-Quantification_Stitches/Original_Scale/Lung/'

# Initialize a dataframe
df = pd.DataFrame(columns=['Date', 
                           'Size Threshold', 
                           'Gaussian Size', 
                           'Truncation', 
                           'Intensity Threshold', 
                           'Minimum Size', 
                           'Virus', 
                           'Animal', 
                           'Replicate', 
                           'Image', 
                           'Count',
                           'Cells Quantified',
                           'Brightness List'])

# Loop through all of the images 
for virus in ['AAV5', 'AAV9', 'CAPA4']:
    for animal in ['1', '2', '3', '4', '5', '6']:
        for replicate in ['1', '2']:
            # Initialize the filename and read in the image
            fname = data_dir + virus + '_' + animal + '_' + replicate + '.TIF'

            im = skimage.io.imread(fname)
            
            # Run the cell countin and brightness function
            binary_mask, n_cells, cell_list, cell_brightness_list = cell_counter(im, template, size_thresh=size_thresh, gaussian_size=gaussian_size, truncation=truncation, threshold=threshold, min_size=min_size)

            # Write all the information into a tidy dataframe
            df = df.append({'Date' : '2019-09-12',
                            'Size Threshold' : size_thresh,
                            'Gaussian Size' : gaussian_size,
                            'Truncation' : truncation,
                            'Intensity Threshold' : threshold, 
                            'Minimum Size' : min_size,
                            'Virus' : virus, 
                            'Animal' : animal, 
                            'Replicate' : replicate, 
                            'Image': virus + '_' + animal + '_' + replicate, 
                            'Count' : int(n_cells),
                            'Cells Quantified' : cell_list, 
                            'Brightness List': cell_brightness_list},
                           ignore_index=True)
            
            # Save the dataframe
            df.to_csv('Lung_Paper_Images/Post-Quantification/Lung/20190912_Stitched_Lung_Cell_Counts_with_Brightness_sampled.csv', index=False)
            
#             # Save the image file as a PNG
#             im_float = skimage.img_as_float(im)[:,:,:2]

#             im_height = np.shape(im)[0]
#             im_width = np.shape(im)[1]

#             template_height = np.shape(template)[0]
#             template_width = np.shape(template)[1]

#             # Build RGB image
#             im_p_rgb = np.dstack((im_float[:,:,0][int(template_height/2 - 1):int(im_height - int(template_height/2)), int(template_width/2 - 1):int(im_width - int(template_width/2))], im_float[:,:,1][int(template_height/2 - 1):int(im_height - int(template_height/2)), int(template_width/2 - 1):int(im_width - int(template_width/2))], binary_mask))
            
#             counted_filename = 'Lung_Paper_Images/Post-Quantification/Testes/' + virus + '_' + animal + '_' + replicate + '_counted.png'
            
#             skimage.io.imsave(counted_filename, im_p_rgb)